In [1]:
import sqlite3
import pandas as pd
from flask import Flask
from flask import Markup
from flask import Flask, request, render_template,send_file, Response, redirect, url_for, jsonify, send_file, session
from werkzeug.wrappers import Request, Response
from werkzeug.serving import run_simple
import json
from dateutil.parser import parse
import os
import database_helper as db_helper
import csvScript
import mySMTP as email
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user, fresh_login_required
from urllib.parse import urlparse, urljoin
from werkzeug.security import generate_password_hash, check_password_hash

In [7]:
csvScript.runScript()

student [caldwell,ivan alexander] is already in the student table
student [robenalt, evan] is already in the student table
student [chua,andreana] is already in the student table
Student [Caldwell,Ivan Alexander] already in Mentoring table. (student has already been assigned).


Student [Robenalt, Evan] already in Mentoring table. (student has already been assigned).
Student [Chua,Andreana] already in Mentoring table. (student has already been assigned).


In [ ]:
app = Flask(__name__)

app.config['TEMPLATES_AUTO_RELOAD'] = True			# If template change, there is no need to reload the APP
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0			# No caching, only for debugging purposes
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
current_user_global = ""

THIS_FOLDER = os.path.abspath('')
db_file = os.path.join(THIS_FOLDER, 'login.db')   
db_file = "sqlite:///" + db_file
fileStr = db_file

app.config['SQLALCHEMY_DATABASE_URI'] = fileStr
app.config['SECRET_KEY'] = 'thissecret'
#app.config['USE_SESSION_FOR_NEXT'] = True

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
login_manager.login_message = "You need to login!"
login_manager.refresh_view ='login'
login_manager.need_refresh_message = 'You need to re-log in to refresh this page'

class User(UserMixin,db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(30),unique=True)
    password = db.Column(db.String(80))
    
def is_safe_url(target):
    ref_url = urlparse(request.host_url)
    test_url = urlparse(urljoin(request.host_url,target))
    return test_url.scheme in ('http','https') and \
        ref_url.netloc == test_url.netloc  


@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))    
      
@app.route('/')
def index():
	return redirect(url_for("login"))

@app.route('/login')
def login():
    if(current_user.is_authenticated == False):
        session['next'] = request.args.get('next', "/main_menu" )
        return render_template('login.html')        
    else:
        return redirect(url_for("mainMenu"))

@app.route('/login_request',methods=['POST','GET'])
def login_request():
    username = request.form['username']
    form_password = request.form['password']
    #print("username: " + form_password)
    #print("form_password: " + form_password)
    user = User.query.filter_by(username=username).first()
    if not user:
        return jsonify({'RESULT':'fail','MSG':'There is no such user.'})
    
    if check_password_hash(user.password, form_password):
        login_user(user,remember=True)
    
        if 'next' in session:
            next = session['next']
            
            if is_safe_url(next):
                return jsonify({'RESULT':'SUCCESS','MSG':next})
            else:
                return jsonify({'RESULT':'SUCCESS','MSG':'/main_menu'})
        else:
            return jsonify({'RESULT':'SUCCESS','MSG':'/main_menu'})
    else:
        return jsonify({'RESULT':'FAIL','MSG':'password incorrect.'})
        

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for("login"))


# A route to show all numbers and the tables including the joined table
@app.route('/showall',methods = ['POST', 'GET'])
@login_required
def showall():
	return render_template("showall.html",
	mentoring_tables=[db_helper.readDatabaseIntoDataframe(connection).to_html(classes='data',index=False)], 
	student_table=[db_helper.readTableIntoDataFrame(connection,"student").to_html(classes='data',index=False)],
	professor_table=[db_helper.readTableIntoDataFrame(connection,"professor").to_html(classes='data',index=False)],
	mentor_table=[db_helper.readTableIntoDataFrame(connection,"mentoring").to_html(classes='data',index=False)],
	offline_data_table=[db_helper.readTableIntoDataFrame(connection,'offline_data').to_html(classes='data',index=False)], 
	most_recent_mentoring_updates_table=[db_helper.readTableIntoDataFrame(connection,'most_recent_mentoring_updates').to_html(classes='data',index=False)],                     
	numberForEachTable=db_helper.getAllNumbers(cursor),
	professorDictionary=db_helper.getProfessorAndStudentNumberInDicionary(cursor))

# The main route for this APP
@app.route('/main_menu',methods = ['POST', 'GET'])
@login_required
def mainMenu():
	professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
	professorJsonString = ""
	for professorName,count in sorted(professorAndNumberDict.items()):
		status = ""
		if count < db_helper.getAllNumbers(cursor)[3]:
			status = "EMPTY"
		elif count >= db_helper.getAllNumbers(cursor)[3] and count < db_helper.getAllNumbers(cursor)[4]:
			status = "ALMOST FULL"
		else:
			status = "FULL"
		professorJsonString += (json.dumps({
			"professorName":professorName.strip().title(),
			"count":count,
			'status': status
		})) + ","
	professorJsonString = '[' + professorJsonString[:-1] + ']'

	student_table = db_helper.readTableIntoDataFrame(connection,"student").sort_values(by=['lastname'])
	student_table['duplicated_name'] = student_table.duplicated(subset=['lastname','firstname'], keep=False)
	studentString = ""
	for i in range(student_table.shape[0]):
		duplicated = 0
		if student_table.iloc[i].duplicated_name == True:
			duplicated = 1
		studentString += (json.dumps({
			"studentName":student_table.iloc[i].lastname.title() + ", " + student_table.iloc[i].firstname.title(),
			"relationship_id":db_helper.findStudentIdByRelationshipId(cursor,student_table.iloc[i].ID),
			'email': student_table.iloc[i].email,
            'duplicated': str(duplicated)
		})) + ","
	studentString = '[' + studentString[:-1] + ']'    
    
	current_user_name  = ""
	if(current_user.is_authenticated == True):
		current_user_name = current_user.username

	return render_template("main_menu.html",
                           studentJson=studentString, 
                           professorJson=professorJsonString, 
                           numberForEachTable=db_helper.getAllNumbers(cursor), current_user_name = current_user_name)

# Update the main_menu page by selecting the element "student_name_input"
@app.route('/update',methods=['POST'])
def update():
	relationship_id = request.form['relationship_id'] 
	professorid = db_helper.findProfessorIdByMentoringId(cursor,relationship_id)
	if professorid is not 'null':
		professorname =  db_helper.getProfessorObjectById(cursor,professorid).lastname
	else:
		professorname =  "None"
	
	studentCountForThisProfessor = db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname]
	if studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[3]:
		color = "green"
	elif studentCountForThisProfessor >= db_helper.getAllNumbers(cursor)[3] and studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[4]:
		color = "yellow"
	else:
		color = "red"
		   
	return jsonify({'RESULT':'SUCCESS','professor_name':professorname.capitalize(),'student_count':db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname],'number_color':color})

# Update the main_menu page by selecting the element "mentor_name_input"
@app.route('/update_from_professor_datalist',methods=['POST'])
def update_from_professor_datalist():
	selected = request.form['name']	
	professorname = selected.strip().lower()
	studentCountForThisProfessor = db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname]
	if studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[3]:
		color = "green"
	elif studentCountForThisProfessor >= db_helper.getAllNumbers(cursor)[3] and studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[4]:
		color = "yellow"
	else:
		color = "red"
		   
	return jsonify({'RESULT':'SUCCESS','student_count':db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname],'number_color':color})


# UPDATE the mentoring table by the assign button
@app.route('/assign_mentor_existed_student',methods=['POST','GET'])
def assign_mentor_existed_student():
	studentId = globalMentoring.student_id
	professorId = globalMentoring.professor_id
	studentName = globalMentoring.studentName
	mentorName = globalMentoring.professorName
	assignDate = globalMentoring.assignedDate
	endDate = globalMentoring.endDate
	relationship_id = globalMentoring.relationship_id
	print("student " + studentName + " is assinged to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
	db_helper.assignExistedStudentToProfessorById(connection,studentId,professorId,assignDate,endDate)
	db_helper.insertDataToMost_recent_mentoring_updatesTable(connection,relationship_id)    
	return jsonify({'RESULT':'ASSIGNED'})

@app.route('/assign_mentor_existed_student_check',methods=['POST','GET'])
def assign_mentor_existed_student_check():
	studentName = request.form['studentName'].lower()
	mentorName = request.form['mentorName'].lstrip().rstrip().lower()
	assignDate = request.form['assignDate']
	endDate = request.form['endDate']
	relationship_id = request.form['relationship_id']
	print("checking condition for: student " + studentName + " is assinging to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
	
	if len(studentName.split(',')) < 2:
		return jsonify({'RESULT':'Student name is not in correct format. For example: "Lastname, Firstname" '})
	
	studentId = db_helper.findStudentIdByRelationshipId(cursor,relationship_id)
	professorId = db_helper.findProfessorIDByName(cursor,mentorName,"")
	
	if studentId == "null":
		return jsonify({'RESULT':'Unable to find this student. Try create new student.'})
	
	if professorId == "null":
		return jsonify({'RESULT':"Unable to find this mentor. Please double check this mentor's name."})
	
	if professorId == db_helper.findProfessorByStudentId(cursor,studentId):
		return jsonify({'RESULT':'This student has already been assigned to this mentor.'})  
    
	globalMentoring.professor_id = professorId
	globalMentoring.student_id = studentId
	globalMentoring.assignedDate = assignDate
	globalMentoring.endDate = endDate
	globalMentoring.studentName = studentName
	globalMentoring.professorName = mentorName
	globalMentoring.relationship_id = relationship_id
    
	return jsonify({'RESULT':'SUCCESS'})
    
    
@app.route('/assign_mentor_new_stduent',methods=['POST','GET'])
def assign_mentor_new_stduent():
	professorId = globalMentoring.professor_id
	studentName = globalMentoring.studentName
	mentorName = globalMentoring.professorName
	assignDate = globalMentoring.assignedDate
	endDate = globalMentoring.endDate
	studentRebelMail = globalMentoring.studentRebelMail
    
	studentLastName = studentName.split(',')[0].rstrip().lstrip()
	studentFirstName = studentName.split(',')[1].rstrip().lstrip()
    
	print("New student " + studentName + " is assinged to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
	studentId = db_helper.insertNewStudent(cursor,studentLastName,studentFirstName,studentRebelMail)
	print("Student id from insertNewStudent: " + str(studentId))
	db_helper.assignNewStudentToProfessorById(connection,studentId,professorId,assignDate,endDate)
	return jsonify({'RESULT':'ASSIGNED'})    

@app.route('/assign_mentor_new_stduent_check',methods=['POST','GET'])
def assign_mentor_new_stduent_check():
    studentLastName = request.form['studentLastName'].rstrip().lstrip().lower()
    studentFirstName = request.form['studentFirstName'].rstrip().lstrip().lower()
    mentorName = request.form['mentorName'].rstrip().lstrip().lower()
    assignDate = request.form['assignDate']
    endDate = request.form['endDate']    
    studentRebelMail = request.form['studentRebelMail'].rstrip().lstrip().lower()
    studentName = studentLastName + ", " + studentFirstName

    print("Checking conditions for student  " + studentName + " is assinging to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
    
    student_id = db_helper.findStudentIDByNameAndMail(cursor,studentLastName,studentFirstName,studentRebelMail)
    if student_id != "null":
        return jsonify({'RESULT':"Student already existed. Try turn off the New Student Switch."})

    professorId = db_helper.findProfessorIDByName(cursor,mentorName,"")       
    if professorId == "null":
        return jsonify({'RESULT':"Unable to find this mentor. Please double check this mentor's name."})    

    globalMentoring.professor_id = professorId
    globalMentoring.assignedDate = assignDate
    globalMentoring.endDate = endDate
    globalMentoring.studentName = studentName
    globalMentoring.professorName = mentorName 
    globalMentoring.studentRebelMail = studentRebelMail
    
    return jsonify({'RESULT':'SUCCESS'})

# No caching, debugging purposes
@app.after_request
def add_header(r):
	"""
	Add headers to both force latest IE rendering engine or Chrome Frame,
	and also to cache the rendered page for 10 minutes.
	"""
	r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
	r.headers["Pragma"] = "no-cache"
	r.headers["Expires"] = "0"
	r.headers['Cache-Control'] = 'public, max-age=0'
	return r

@app.route('/mentoring_joined',methods=['POST','GET'])
@login_required
def mentoring_joined():
	most_recent_changes_json_string = json.dumps(db_helper.getMost_recent_mentoring_updatesTableAsDataframeForFlask(connection).sort_index(ascending=False).values.tolist())
	professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
	newString = ""
	for professorName,count in sorted(professorAndNumberDict.items()):
		newString += (json.dumps({
			"professorName":professorName.strip().title(),
			"count":count
		})) + ","
	newString = '[' + newString[:-1] + ']'
    
	try:
		studentName = request.args.get('username').title()
	except:
		studentName = ""
        
	current_user_name  = ""
	if(current_user.is_authenticated == True):
		current_user_name = current_user.username
        
	return render_template("mentoring_joined.html",
		mentoring_joined_tables=[db_helper.readDatabaseIntoDataframe(connection,True).to_html(classes='data',index=False)],
		studentName = studentName,
		professorJson=newString,
		most_recent_changes_json_string = most_recent_changes_json_string, current_user_name = current_user_name                   
      )
  
@app.route('/apply_settings',methods=['POST'])   
def apply_settings():
    margin = float(request.form['margin'])
    db_helper.updateDataFromOffline_dataTable(connection,'DOUBLE_margin_for_max_student_count',margin)
    return jsonify({'RESULT':'SUCCESS'}) 

@app.route('/backup_database',methods=['POST','GET'])
@login_required
def backup_database():
	current_user_name  = ""
	if(current_user.is_authenticated == True):
		current_user_name = current_user.username       
        
	return render_template("backup_database.html",current_user_name=current_user_name) 

@app.route('/request_for_backup_database')   
def request_for_backup_database():
    THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))
    db_file = os.path.join(THIS_FOLDER, "falcuty_mentor.db")
    try:
        file = send_file(db_file, as_attachment=True,attachment_filename='UNLV_CS_mentor.db') 
    except Exception as e:
        file = str(e)
    
    return file


@app.route('/multiple_assign_page',methods=['POST','GET'])
@login_required
def multiple_assign_page():
	professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
	newString = ""
	for professorName,count in sorted(professorAndNumberDict.items()):
		newString += (json.dumps({
			"professorName":professorName.strip().title(),
			"count":count
		})) + ","
	newString = '[' + newString[:-1] + ']'
	studentForProfessorsJson = db_helper.getStudentsForProfessor(connection)
    
	current_user_name  = ""    
	if(current_user.is_authenticated == True):
		current_user_name = current_user.username    
    
	return render_template("multiple_assign_page.html",professorJson=newString,studentForProfessorsJson=studentForProfessorsJson,current_user_name=current_user_name)

@app.route('/multiple_assign_request',methods=['POST','GET'])
def multiple_assign_request():
    teacherName = request.form['teacherName']
    relationshipIdArray = request.form.getlist('relationshipIdArray[]')
    db_helper.massAssign(connection,relationshipIdArray,teacherName)
    return jsonify({'RESULT':'SUCCESS'})

# Running the app here
if __name__ == '__main__':
	connection = sqlite3.connect("falcuty_mentor.db")
	connection.execute("PRAGMA foreign_keys = ON;")
	cursor = connection.cursor()
	globalMentoring = db_helper.Mentoring()
	run_simple('localhost', 5000, app,use_debugger=True, use_evalex=True)


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2020 00:54:15] "GET /main_menu HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 00:54:15] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:15] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:15] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:20] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 00:54:20] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:20] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:20] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:21] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 00:54:22] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 00:54:22] "GET /static/css/util.css HTTP/1.

127.0.0.1 - - [29/Mar/2020 01:12:00] "GET /static/fonts/OpenSans/OpenSans-Regular.ttf HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:13] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:12:13] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:13] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:13] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:13] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:15] "GET /backup_database HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:12:15] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:15] "GET /static/jquery.flexdatalist.min2.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:12:16] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:12:16] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/M

127.0.0.1 - - [29/Mar/2020 01:22:56] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:22:56] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:22:56] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:22:56] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:22:56] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:25:35] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:25:35] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:25:36] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:25:36] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:25:36] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:26:12] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:26:12]

assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "22";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "23";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "24";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "72";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "73";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "74";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "25";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/29/2020" WHERE ID = "75";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "03/

127.0.0.1 - - [29/Mar/2020 01:26:46] "POST /multiple_assign_request HTTP/1.1" 200 -


...

127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:01] "GET /static/fonts/OpenSans/OpenSans-Regular.ttf HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:22] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:27:22] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:22] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:22] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:22] "GET /static/im

assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "1";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "2";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "3";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "4";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "5";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "6";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "7";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "13";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "0

127.0.0.1 - - [29/Mar/2020 01:27:31] "POST /multiple_assign_request HTTP/1.1" 200 -


assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "8";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "16";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "20";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "15";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "19", dateAssigned = "03/29/2020" WHERE ID = "19";


127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:27:47] "GET /static/fonts/OpenSans/OpenSans-Regular.ttf HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:28:18] "GET /backup_database HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:28:18] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:28:18] "GET /static/jquery.flexdatalist.min2.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2020 01:28:22] "GET /main_menu HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2020 01:28:22] "GET /static/jquery.flexdatal

SyntaxError: invalid syntax (<ipython-input-16-e172bc12abfe>, line 1)

In [ ]:
import database_helper as db_helper
import importlib
importlib.reload(db_helper)
connection = sqlite3.connect("falcuty_mentor.db")
connection.execute("PRAGMA foreign_keys = ON;")
cursor = connection.cursor()
globalMentoring = db_helper.Mentoring()

In [ ]:
import datetime
currentDT = datetime.datetime.now()
currentDTString = currentDT.strftime("%m/%d/%Y")

#db_helper.insertDataToMost_recent_mentoring_updatesTable(connection,"55",currentDTString)
#db_helper.insertDataToMost_recent_mentoring_updatesTable(connection,"87",currentDTString)
#db_helper.insertDataToMost_recent_mentoring_updatesTable(connection,"108","03/26/2020 13:25:34")